In [ ]:
from PIL import Image
import numpy as np

def message_to_bin(message):
    binary = ''.join(format(ord(i), '08b') for i in message)
    return binary

def modify_pixel(pixel, binary_message, index):
    if index < len(binary_message):
        pixel = (pixel & ~1) | int(binary_message[index])
    return pixel

def encode_image(img_path, message):
    img = Image.open(img_path)
    binary_message = message_to_bin(message) + '11111111'  # Delimiter to indicate end of message
    pixels = np.array(img)

    index = 0
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            for k in range(3):  # Iterate over RGB channels
                if index < len(binary_message):
                    pixels[i, j, k] = modify_pixel(pixels[i, j, k], binary_message, index)
                    index += 1

    encoded_img = Image.fromarray(pixels)
    return encoded_img

# Example usage:
encoded_image = encode_image('input_image.png', 'YourSecretCode123')
encoded_image.save('encoded_image.png')


In [ ]:
def decode_image(img_path):
    img = Image.open(img_path)
    pixels = np.array(img)11
    binary_message = ''
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            for k in range(3):
                binary_message += str(pixels[i, j, k] & 1)
                if binary_message[-8:] == '11111111':  # Check for delimiter
                    return ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message)-8, 8))
    return None

# Example usage:
decoded_message = decode_image('encoded_image.png')
print(decoded_message)


YourSecretCode123


In [ ]:
from PIL import Image
import numpy as np

def add_noise(img, mean=0, var=0.01):
    """ Add Gaussian noise to an image. """
    row, col, ch = img.shape
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = img + gauss
    return np.clip(noisy, 0, 255).astype(np.uint8)

def crop_image(img, start_x, start_y, end_x, end_y):
    """ Crop an image. """
    return img[start_y:end_y, start_x:end_x]

def rotate_image(img, angle):
    """ Rotate an image. """
    return img.rotate(angle, expand=True)

def tamper_image(img_path, save_path='tampered_image.png'):
    img = Image.open(img_path)
    img = np.array(img)

    # Optionally add noise
    img = add_noise(img)

    # Optionally crop the image (specify the crop area)
    #img = crop_image(img, 50, 50, 200, 200)  # Adjust these values based on your image dimensions

    # Convert back to PIL Image to rotate
    img = Image.fromarray(img)
    #img = rotate_image(img, 45)  # Rotate by 45 degrees

    # Save the tampered image
    img.save(save_path)
    return save_path

# Example usage:
tampered_image_path = tamper_image('input_image.png')
print("Tampered image saved to:", tampered_image_path)


Tampered image saved to: tampered_image.png


### ResNet50

In [ ]:
from PIL import Image
import numpy as np

def message_to_bin(message):
    binary = ''.join(format(ord(i), '08b') for i in message)
    return binary

def modify_pixel(pixel, binary_message, index):
    if index < len(binary_message):
        pixel = (pixel & ~1) | int(binary_message[index])
    return pixel

def encode_image(img_path, message, save_path='encoded_image.png'):
    img = Image.open(img_path)
    binary_message = message_to_bin(message) + '11111111'  # Delimiter to indicate end of message
    pixels = np.array(img)

    index = 0
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            for k in range(3):  # Iterate over RGB channels
                if index < len(binary_message):
                    pixels[i, j, k] = modify_pixel(pixels[i, j, k], binary_message, index)
                    index += 1

    encoded_img = Image.fromarray(pixels)
    encoded_img.save(save_path)
    return save_path


import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'encoded' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = ImageDataset('/content/', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained model and modify it for binary classification
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.9581713080406189
Epoch 2, Loss: 0.8205904364585876
Epoch 3, Loss: 0.6710663437843323
Epoch 4, Loss: 0.6272410154342651
Epoch 5, Loss: 0.6771294474601746
Epoch 6, Loss: 0.7419540882110596
Epoch 7, Loss: 0.7706840634346008
Epoch 8, Loss: 0.7510538101196289
Epoch 9, Loss: 0.6946001052856445
Epoch 10, Loss: 0.6298009753227234
Accuracy: 66.66666666666667%


In [ ]:
predicted

tensor([0, 0, 0])

### VGG16

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'encoded' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = ImageDataset('/content/', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained VGG16 model and modify it for binary classification
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)  # Modify the classifier layer

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


Epoch 1, Loss: 0.4455904960632324
Epoch 2, Loss: 0.5198726058006287
Epoch 3, Loss: 0.5281772613525391
Epoch 4, Loss: 0.9445035457611084
Epoch 5, Loss: 0.7109470367431641
Accuracy: 100.0%


In [ ]:
labels

tensor([0, 0, 1])

### Inception_v3

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'encoded' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations for Inception v3
transform = transforms.Compose([
    transforms.Resize(299),  # Resize so the smallest side is 299 pixels
    transforms.CenterCrop(299),  # Crop to 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = ImageDataset('/content/', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained Inception v3 model and modify it for binary classification
model = models.inception_v3(pretrained=True)
# Handle the auxiliary net
aux_in_features = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(aux_in_features, 2)
# Handle the primary net
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs, aux_outputs = model(inputs)  # Inception model outputs main and auxiliary outputs
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2  # Combine losses
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs, _ = model(inputs)  # Ignore auxiliary outputs during evaluation
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


Epoch 1, Loss: 0.9869362115859985
Epoch 2, Loss: 0.9992379546165466
Epoch 3, Loss: 0.8710058331489563
Epoch 4, Loss: 0.8620637059211731
Epoch 5, Loss: 0.7637868523597717
Accuracy: 66.66666666666667%


In [ ]:
labels

tensor([0, 1, 0])

#### HIDDEN: 50 images each

In [ ]:
import os
import random
import shutil

# Define source directories
image_dir = '/content/dataset/ukraine_war_resized'
tampered_dir = '/content/dataset/ukraine_war_tampered'

# Define destination directory
destination_dir = '/content/dataset/final_100'

# Function to copy a subset of images from source directory to destination directory
def copy_subset_images(source_dir, dest_dir, num_images):
    # Get a list of all files in the source directory
    files = os.listdir(source_dir)
    # Shuffle the list
    random.shuffle(files)
    # Take a subset of the shuffled list
    subset = files[:num_images]
    # Copy each file from the subset to the destination directory
    for filename in subset:
        shutil.copy(os.path.join(source_dir, filename), dest_dir)

# Copy subset of original images
copy_subset_images(image_dir, destination_dir, 50)

# Copy subset of tampered images
copy_subset_images(tampered_dir, destination_dir, 50)

print("Subset of 50 original and 50 tampered images copied successfully to", destination_dir)

# Get the list of files in the directory
files = os.listdir(destination_dir)

# Count the number of files with "tampered" in their name
tampered_count = sum('tampered' in filename for filename in files)

print("Number of files with 'tampered' in their name:", tampered_count)

Subset of 50 original and 50 tampered images copied successfully to /content/dataset/final_100
Number of files with 'tampered' in their name: 50


In [ ]:
copy_to_drive('/content/dataset/final_100','/content/drive/MyDrive/Courses/Sem 2/DL/final_100')

Folder copied successfully to Google Drive.


### Entire Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import gdown
import zipfile
import os

# Define the URL of the ZIP file
zip_url = 'https://drive.google.com/uc?id=1XBrjRp_1ssW_QqmrGbO3kO6vQm8tlUck'

# Define the directory where you want to extract the ZIP file
extracted_dir_path = '/content/dataset/ukraine_war'

# Create the directory if it doesn't exist
os.makedirs(extracted_dir_path, exist_ok=True)

# Download the ZIP file
zip_file_path = os.path.join(extracted_dir_path, 'ukraine_war.zip')
gdown.download(zip_url, zip_file_path, quiet=False)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

# Check the contents of the extracted directory
extracted_files = os.listdir(extracted_dir_path)
print("Contents of the extracted directory:")
print(extracted_files)


Downloading...
From (original): https://drive.google.com/uc?id=1XBrjRp_1ssW_QqmrGbO3kO6vQm8tlUck
From (redirected): https://drive.google.com/uc?id=1XBrjRp_1ssW_QqmrGbO3kO6vQm8tlUck&confirm=t&uuid=e0662617-a24c-4935-91bb-0008d692052a
To: /content/dataset/ukraine_war/ukraine_war.zip
100%|██████████| 1.58G/1.58G [00:27<00:00, 56.4MB/s]


Contents of the extracted directory:
['ukraine_war.zip', 'ukraine_war']


In [ ]:
import os
import cv2

def convert_and_resize_images(image_dir, output_dir, target_size=(224, 224)):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate through each image in the directory
    for filename in os.listdir(image_dir):
        img_path = os.path.join(image_dir, filename)
        try:
            # Read the image
            img = cv2.imread(img_path)

            # Convert the image to RGB format
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Resize the image
            img = cv2.resize(img, target_size)

            # Save the image as JPG format
            output_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '.jpg')
            cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

            print(f"Image '{img_path}' converted and resized successfully.")
        except Exception as e:
            print(f"Error processing image '{img_path}': {e}")

# Define input and output directories
image_dir = '/content/dataset/ukraine_war/ukraine_war'
output_dir = '/content/dataset/ukraine_war_resized'

# Convert and resize images
convert_and_resize_images(image_dir, output_dir)

import os
import cv2

def check_images(directory):
    image_formats = set()
    image_sizes = set()
    num_images = 0

    # Iterate through each file in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        # Check if the file is an image
        if os.path.isfile(filepath) and any(filename.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
            # Increment the count of images
            num_images += 1
            # Read the image using OpenCV
            image = cv2.imread(filepath)
            if image is not None:
                # Get the image format
                image_format = filename.split('.')[-1]
                image_formats.add(image_format)
                # Get the image size
                image_size = tuple(image.shape[1::-1])
                image_sizes.add(image_size)
            else:
                print(f"Error: Unable to read image '{filename}'")

    # Print the results
    print(f"Number of images: {num_images}")
    print(f"Image formats found: {image_formats}")
    print(f"Image sizes found: {image_sizes}")

# Define the directory containing the images
image_directory = '/content/dataset/ukraine_war_resized'

# Call the function to check images
check_images(image_directory)


Image '/content/dataset/ukraine_war/ukraine_war/4fh.png.4' converted and resized successfully.
Image '/content/dataset/ukraine_war/ukraine_war/6j.jpg' converted and resized successfully.
Image '/content/dataset/ukraine_war/ukraine_war/437.png' converted and resized successfully.
Image '/content/dataset/ukraine_war/ukraine_war/96.png' converted and resized successfully.
Image '/content/dataset/ukraine_war/ukraine_war/438.png.2' converted and resized successfully.
Image '/content/dataset/ukraine_war/ukraine_war/39.png' converted and resized successfully.
Image '/content/dataset/ukraine_war/ukraine_war/b55.jpg' converted and resized successfully.
Image '/content/dataset/ukraine_war/ukraine_war/559.jpg' converted and resized successfully.
Image '/content/dataset/ukraine_war/ukraine_war/332.jpg' converted and resized successfully.
Image '/content/dataset/ukraine_war/ukraine_war/f22.png' converted and resized successfully.
Image '/content/dataset/ukraine_war/ukraine_war/541.png' converted an

#### Compare if original and encoded images are lossy or lossless

In [ ]:
import os
from PIL import Image
import numpy as np

def message_to_bin(message):
    binary = ''.join(format(ord(i), '08b') for i in message)
    return binary

def modify_pixel(pixel, binary_message, index):
    if index < len(binary_message):
        pixel = (pixel & ~1) | int(binary_message[index])
    return pixel

def encode_image(img_path, message):
    img = Image.open(img_path)
    binary_message = message_to_bin(message) + '11111111'  # Delimiter to indicate end of message
    pixels = np.array(img)

    index = 0
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            for k in range(3):  # Iterate over RGB channels
                if index < len(binary_message):
                    pixels[i, j, k] = modify_pixel(pixels[i, j, k], binary_message, index)
                    index += 1

    encoded_img = Image.fromarray(pixels)
    return encoded_img

def compare_file_sizes(original_img_path, encoded_img_path):
    original_size = os.path.getsize(original_img_path)
    encoded_size = os.path.getsize(encoded_img_path)
    print("Original image size:", original_size, "bytes")
    print("Encoded image size:", encoded_size, "bytes")
    if original_size == encoded_size:
        print("The encoded image size is the same as the original image size. It suggests lossless encoding.")
    else:
        print("The encoded image size is different from the original image size. It suggests lossy encoding.")

# Usage
original_img_path = "/content/dataset/ukraine_war/ukraine_war/07.png"
encoded_img_path = "/content/07encoded_image.jpg"
message = "Your secret message here"

# Encoding the image
encoded_img = encode_image(original_img_path, message)
encoded_img.save(encoded_img_path)

# Compare file sizes
compare_file_sizes(original_img_path, encoded_img_path)

def compare_images(img_path1, img_path2):
    img1 = Image.open(img_path1)
    img2 = Image.open(img_path2)

    pixels1 = np.array(img1)
    pixels2 = np.array(img2)

    if np.array_equal(pixels1, pixels2):
        print("The pixel values of the original and encoded images are exactly the same. Lossless encoding.")
    else:
        print("The pixel values of the original and encoded images are different. Lossy encoding.")

# Usage
original_img_path = "/content/dataset/ukraine_war/ukraine_war/07.png"
encoded_img_path = "/content/07encoded_image.jpg"

# Compare pixel values
compare_images(original_img_path, encoded_img_path)

Original image size: 1182471 bytes
Encoded image size: 128945 bytes
The encoded image size is different from the original image size. It suggests lossy encoding.
The pixel values of the original and encoded images are different. Lossy encoding.


In [ ]:
from PIL import Image
import numpy as np

def add_noise(img, mean=0, var=0.01):
    """ Add Gaussian noise to an image. """
    row, col, ch = img.shape
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = img + gauss
    return np.clip(noisy, 0, 255).astype(np.uint8)

def crop_image(img, start_x, start_y, end_x, end_y):
    """ Crop an image. """
    return img[start_y:end_y, start_x:end_x]

def rotate_image(img, angle):
    """ Rotate an image. """
    return img.rotate(angle, expand=True)

def tamper_image_path(img_path, save_path='tampered_image.png'):
    img = Image.open(img_path)
    img = np.array(img)

    # Optionally add noise
    img = add_noise(img)

    # Optionally crop the image (specify the crop area)
    #img = crop_image(img, 50, 50, 200, 200)  # Adjust these values based on your image dimensions

    # Convert back to PIL Image to rotate
    img = Image.fromarray(img)
    #img = rotate_image(img, 45)  # Rotate by 45 degrees

    return img

def tamper_images_data(image_dir, output_dir, target_size=(224, 224)):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    count = 0
    # Iterate through each image in the directory
    for filename in os.listdir(image_dir):
        if count==1400:
          break
        img_path = os.path.join(image_dir, filename)
        try:

            img = tamper_image_path(img_path)

            # Save the image as JPG format
            output_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '_tampered.jpg')
            img.save(output_path)

            print(f"Image '{img_path}' tampered successfully.")
            count+=1
        except Exception as e:
            print(f"Error processing image '{img_path}': {e}")

# Define input and output directories
image_dir = '/content/dataset/ukraine_war_resized'
output_dir = '/content/dataset/ukraine_war_tampered'

tamper_images_data(image_dir,output_dir)

Image '/content/dataset/ukraine_war_resized/4313.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/6j.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/b55.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/559.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/332.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/4466.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/f68.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/13.png.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/5548.png.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/b7.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/n5.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/570.png.jpg' tampered successfully.
Image '/content/dataset/ukraine_war_resized/57j.jpg' tampered successfully.
I

In [ ]:
image_directory = '/content/dataset/ukraine_war_tampered'

# Call the function to check images
check_images(image_directory)
compare_file_sizes('/content/dataset/ukraine_war_resized/07.jpg', '/content/dataset/ukraine_war_tampered/07_tampered.jpg')
compare_images('/content/dataset/ukraine_war_resized/07.jpg', '/content/dataset/ukraine_war_tampered/07_tampered.jpg')

Number of images: 1400
Image formats found: {'jpg'}
Image sizes found: {(224, 224)}
Original image size: 35534 bytes
Encoded image size: 16792 bytes
The encoded image size is different from the original image size. It suggests lossy encoding.
The pixel values of the original and encoded images are different. Lossy encoding.


In [ ]:
import os
import shutil

image_dir = '/content/dataset/ukraine_war_resized'
image_dir2 = '/content/dataset/ukraine_war_tampered'
image_final_dir = '/content/dataset/ukraine_war_finaldata'

# Create the final directory if it doesn't exist
if not os.path.exists(image_final_dir):
    os.makedirs(image_final_dir)

# Copy images from image_dir to image_final_dir
for filename in os.listdir(image_dir):
    shutil.copy(os.path.join(image_dir, filename), image_final_dir)

# Copy images from image_dir2 to image_final_dir
for filename in os.listdir(image_dir2):
    shutil.copy(os.path.join(image_dir2, filename), image_final_dir)

print("Images copied successfully to", image_final_dir)
check_images(image_final_dir)

Images copied successfully to /content/dataset/ukraine_war_finaldata
Number of images: 3083
Image formats found: {'jpg'}
Image sizes found: {(224, 224)}


In [ ]:
def copy_to_drive(colab_folder_path, drive_folder_path):
   colab_folder_path = '/content/dataset/ukraine_war_finaldata'
   drive_folder_path = '/content/drive/MyDrive/Courses/Sem 2/DL/ukraine_war_finaldata'

  # Copy the folder from Colab to Drive
  shutil.copytree(colab_folder_path, drive_folder_path)

  print("Folder copied successfully to Google Drive.")

In [ ]:
import os
import random
import shutil

# Define source directories
image_dir = '/content/dataset/ukraine_war_resized'
tampered_dir = '/content/dataset/ukraine_war_tampered'

# Define destination directory
destination_dir = '/content/dataset/final'

# Function to copy a subset of images from source directory to destination directory
def copy_subset_images(source_dir, dest_dir, num_images):
    # Get a list of all files in the source directory
    files = os.listdir(source_dir)
    # Shuffle the list
    random.shuffle(files)
    # Take a subset of the shuffled list
    subset = files[:num_images]
    # Copy each file from the subset to the destination directory
    for filename in subset:
        shutil.copy(os.path.join(source_dir, filename), dest_dir)

# Copy subset of original images
copy_subset_images(image_dir, destination_dir, 200)

# Copy subset of tampered images
copy_subset_images(tampered_dir, destination_dir, 200)

print("Subset of 200 original and 200 tampered images copied successfully to", destination_dir)


Subset of 200 original and 200 tampered images copied successfully to /content/dataset/final


In [ ]:
import os

# Define the directory
destination_dir = '/content/dataset/final'

# Get the list of files in the directory
files = os.listdir(destination_dir)

# Count the number of files with "tampered" in their name
tampered_count = sum('tampered' in filename for filename in files)

print("Number of files with 'tampered' in their name:", tampered_count)

Number of files with 'tampered' in their name: 200


###Inception_v3

#### Non encoded images vs Tampered Images

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations for Inception v3
transform = transforms.Compose([
    transforms.Resize(299),  # Resize so the smallest side is 299 pixels
    transforms.CenterCrop(299),  # Crop to 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = ImageDataset('/content/dataset/final', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained Inception v3 model and modify it for binary classification
model = models.inception_v3(pretrained=True)
# Handle the auxiliary net
aux_in_features = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(aux_in_features, 2)
# Handle the primary net
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs, aux_outputs = model(inputs)  # Inception model outputs main and auxiliary outputs
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2  # Combine losses
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs, _ = model(inputs)  # Ignore auxiliary outputs during evaluation
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.6973464488983154
Epoch 2, Loss: 0.7377987504005432
Epoch 3, Loss: 0.6356834769248962
Epoch 4, Loss: 0.3975643217563629
Epoch 5, Loss: 0.11516178399324417
Epoch 6, Loss: 0.48788732290267944
Epoch 7, Loss: 0.061480358242988586
Epoch 8, Loss: 0.6364486813545227
Epoch 9, Loss: 0.08867169916629791
Epoch 10, Loss: 1.2352858781814575
Accuracy: 91.25%


#### Encoded images with Tampering

In [ ]:
img_dir = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50'
# if not os.path.exists(img_dir):
#     os.makedirs(img_dir)
# copy_subset_images('/content/dataset/ukraine_war_resized', img_dir, 50)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import os

#Load pre-trained ResNet50 model
model = ResNet50(weights='imagenet')

def encode_image(img,message):
  def message_to_bin(message):
      binary = ''.join(format(ord(i), '08b') for i in message)
      return binary

  def modify_pixel(pixel, binary_message, index):
      if index < len(binary_message):
          pixel = (pixel & ~1) | int(binary_message[index])
      return pixel

  def enc_image(img, message):
      binary_message = message_to_bin(message) + '11111111'  # Delimiter to indicate end of message
      pixels = np.array(img)

      index = 0
      for i in range(pixels.shape[0]):
          for j in range(pixels.shape[1]):
              for k in range(3):  # Iterate over RGB channels
                  if index < len(binary_message):
                      pixels[i, j, k] = modify_pixel(pixels[i, j, k], binary_message, index)
                      index += 1

      encoded_img = Image.fromarray(pixels)
      return encoded_img
  return enc_image(img,message)


def decode_image(img):

    pixels = np.array(img)
    binary_message = ''
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            for k in range(3):
                binary_message += str(pixels[i, j, k] & 1)
                if binary_message[-8:] == '11111111':  # Check for delimiter
                    return ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message)-8, 8))
    return None

# # Define the path to your dataset
dataset_path = img_dir

# Iterate through each image in the dataset
for filename in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, filename)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Perform inference
    preds = model.predict(x)
    # Decode and print the top-3 predicted classes
    map = decode_predictions(preds, top=3)[0]
    print(f'Predicted for {filename}:', map[0][1])
    secret = f'TOPSECRET: This is the {map[0][1]}.'
    encoded_img = encode_image(img, secret)
    encoded_img.save(img_path.replace('jpg','png'))
    #print("Decoded: ", decode_image(encoded_img))



1/1 [==============================] - 2s 2s/step
Predicted for c4.jpg: snowplow
1/1 [==============================] - 0s 355ms/step
Predicted for 3234.jpg: tow_truck
1/1 [==============================] - 0s 382ms/step
Predicted for v90.jpg: half_track
1/1 [==============================] - 0s 359ms/step
Predicted for dg.jpg: tank
1/1 [==============================] - 0s 337ms/step
Predicted for br2.jpg: tank
1/1 [==============================] - 0s 370ms/step
Predicted for 3322.jpg: cliff
1/1 [==============================] - 0s 338ms/step
Predicted for vb.jpg: shower_curtain
1/1 [==============================] - 0s 335ms/step
Predicted for 43fhl.jpg: half_track
1/1 [==============================] - 0s 372ms/step
Predicted for 6688.jpg: tank
1/1 [==============================] - 0s 332ms/step
Predicted for q3.jpg: trailer_truck
1/1 [==============================] - 0s 214ms/step
Predicted for 6888.jpg: stove
1/1 [==============================] - 0s 203ms/step
Predicted for S

In [ ]:
from PIL import Image
import numpy as np

def decode_image(img):

    pixels = np.array(img)
    binary_message = ''
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            for k in range(3):
                binary_message += str(pixels[i, j, k] & 1)
                if binary_message[-8:] == '11111111':  # Check for delimiter
                    return ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message)-8, 8))
    return None

def tamper_image(img, save_path='tampered_image.png'):
    img = np.array(img)

    # Optionally add noise
    img = add_noise(img)

    # Optionally crop the image (specify the crop area)
    #img = crop_image(img, 50, 50, 200, 200)  # Adjust these values based on your image dimensions

    # Convert back to PIL Image to rotate
    img = Image.fromarray(img)
    #img = rotate_image(img, 45)  # Rotate by 45 degrees
    return img


#encoded_image = encode_image('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50/312.png', 'YourSecretCode123')
tamper_image = tamper_image(Image.open('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50/312.png'))

decoded_message = decode_image(tamper_image)
print(decoded_message)

ìMË¿g	öFÙZcê¶Þ±¶w·ÀrÜÃHk6MÐPbc´ÎÀ÷_¡ýd©ÃieÌV¾Ã·gÎ9U6@ .=È!j¿


#### Tamper Encoded Data - Encoded_50

In [ ]:
def tamper_images_data(image_dir, output_dir, target_size=(224, 224)):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    count = 0
    # Iterate through each image in the directory
    for filename in os.listdir(image_dir):
        if count==1400:
          break
        img_path = os.path.join(image_dir, filename)
        try:

            img = tamper_image_path(img_path)

            # Save the image as JPG format
            output_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '_tampered.png')
            img.save(output_path)

            print(f"Image '{img_path}' tampered successfully.")
            count+=1
        except Exception as e:
            print(f"Error processing image '{img_path}': {e}")

def tamper_image_path(img_path, save_path='tampered_image.png'):
    img = Image.open(img_path)
    img = np.array(img)

    # Optionally add noise
    img = add_noise(img)

    # Optionally crop the image (specify the crop area)
    #img = crop_image(img, 50, 50, 200, 200)  # Adjust these values based on your image dimensions

    # Convert back to PIL Image to rotate
    img = Image.fromarray(img)
    #img = rotate_image(img, 45)  # Rotate by 45 degrees

    return img
tamper_images_data('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50', '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50')

#### ResNet

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained model and modify it for binary classification
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Epoch 1, Loss: 0.8656717538833618
Epoch 2, Loss: 0.7513283491134644
Epoch 3, Loss: 0.854275643825531
Epoch 4, Loss: 0.9165647029876709
Epoch 5, Loss: 0.6640243530273438
Epoch 6, Loss: 0.7973805665969849
Epoch 7, Loss: 0.7519326210021973
Epoch 8, Loss: 0.7097175717353821
Epoch 9, Loss: 1.1205084323883057
Epoch 10, Loss: 0.5354132652282715
Accuracy: 54.0%


####Inception_v3

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations for Inception v3
transform = transforms.Compose([
    transforms.Resize(299),  # Resize so the smallest side is 299 pixels
    transforms.CenterCrop(299),  # Crop to 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained Inception v3 model and modify it for binary classification
model = models.inception_v3(pretrained=True)
# Handle the auxiliary net
aux_in_features = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(aux_in_features, 2)
# Handle the primary net
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs, aux_outputs = model(inputs)  # Inception model outputs main and auxiliary outputs
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2  # Combine losses
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs, _ = model(inputs)  # Ignore auxiliary outputs during evaluation
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 1.3888630867004395
Epoch 2, Loss: 1.0894582271575928
Epoch 3, Loss: 1.636603832244873
Epoch 4, Loss: 1.6251716613769531
Epoch 5, Loss: 1.0721657276153564
Epoch 6, Loss: 1.2732505798339844
Epoch 7, Loss: 1.1816078424453735
Epoch 8, Loss: 1.679003357887268
Epoch 9, Loss: 0.6254927515983582
Epoch 10, Loss: 1.2098230123519897
Accuracy: 48.0%


#### Vgg16

In [ ]:
# Load the dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained VGG16 model and modify it for binary classification
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)  # Modify the classifier layer

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.6140020489692688
Epoch 2, Loss: 0.720977783203125
Epoch 3, Loss: 0.8592000603675842
Epoch 4, Loss: 0.7762177586555481
Epoch 5, Loss: 0.6702403426170349
Epoch 6, Loss: 0.7327219247817993
Epoch 7, Loss: 0.6631041765213013
Epoch 8, Loss: 0.6718435883522034
Epoch 9, Loss: 0.7308367490768433
Epoch 10, Loss: 0.7235779166221619
Accuracy: 47.0%


#### Encoded and compressed images with Tampering

#### Lossy and Lossless compression- Encoded_50_compressed Lossy/Lossless

In [ ]:
import os
from PIL import Image

# Define source directory
source_dir = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50'

# Define destination directories for lossy and lossless compression
destination_lossy = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossy'
destination_lossless = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossless'

# Create destination directories if they don't exist
os.makedirs(destination_lossy, exist_ok=True)
os.makedirs(destination_lossless, exist_ok=True)

# Iterate over files in the source directory
for filename in os.listdir(source_dir):
    # Check if the filename does not contain 'tampered'
    if 'tampered' not in filename:
        # Construct the full path of the source file
        source_file = os.path.join(source_dir, filename)

        # Open the image
        img = Image.open(source_file)

        # Save the image with lossy compression
        img.save(os.path.join(destination_lossy, filename), quality=80)  # Adjust quality as needed

        # Save the image with lossless compression
        img.save(os.path.join(destination_lossless, filename), format='PNG')

print("Images moved and compressed successfully.")


Images moved and compressed successfully.


#### Tamper Encoded+Compressed Data

In [ ]:
def add_noise(img, mean=0, var=0.01):
    """ Add Gaussian noise to an image. """
    row, col, ch = img.shape
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = img + gauss
    return np.clip(noisy, 0, 255).astype(np.uint8)
tamper_images_data('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossy', '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossy')

tamper_images_data('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossless', '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossless')

#### ResNet for Lossy vs Lossless compression

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the Lossy dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossy', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained model and modify it for binary classification
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossy compression: {100 * correct / total}%')


# Load the Lossless dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossless', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained model and modify it for binary classification
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossless compression: {100 * correct / total}%')

Epoch 1, Loss: 0.6748802661895752
Epoch 2, Loss: 1.2062875032424927
Epoch 3, Loss: 0.8119528293609619
Epoch 4, Loss: 0.6255300045013428
Epoch 5, Loss: 1.1000237464904785
Epoch 6, Loss: 0.867458701133728
Epoch 7, Loss: 0.9236495494842529
Epoch 8, Loss: 1.476010799407959
Epoch 9, Loss: 0.8994950652122498
Epoch 10, Loss: 1.5172618627548218
Accuracy for Lossy compression: 50.0%
Epoch 1, Loss: 1.0058207511901855
Epoch 2, Loss: 0.7604208588600159
Epoch 3, Loss: 1.203510046005249
Epoch 4, Loss: 1.2051016092300415
Epoch 5, Loss: 0.5238816142082214
Epoch 6, Loss: 0.9600765705108643
Epoch 7, Loss: 0.8668262362480164
Epoch 8, Loss: 0.7841525673866272
Epoch 9, Loss: 0.7749594449996948
Epoch 10, Loss: 0.5793909430503845
Accuracy for Lossless compression: 50.0%


####Inception_v3 for Lossy vs Lossless comparison

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations for Inception v3
transform = transforms.Compose([
    transforms.Resize(299),  # Resize so the smallest side is 299 pixels
    transforms.CenterCrop(299),  # Crop to 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the Lossy dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossy', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained Inception v3 model and modify it for binary classification
model = models.inception_v3(pretrained=True)
# Handle the auxiliary net
aux_in_features = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(aux_in_features, 2)
# Handle the primary net
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs, aux_outputs = model(inputs)  # Inception model outputs main and auxiliary outputs
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2  # Combine losses
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs, _ = model(inputs)  # Ignore auxiliary outputs during evaluation
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossy compression: {100 * correct / total}%')


# Load the Lossless dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossless', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained Inception v3 model and modify it for binary classification
model = models.inception_v3(pretrained=True)
# Handle the auxiliary net
aux_in_features = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(aux_in_features, 2)
# Handle the primary net
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs, aux_outputs = model(inputs)  # Inception model outputs main and auxiliary outputs
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2  # Combine losses
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs, _ = model(inputs)  # Ignore auxiliary outputs during evaluation
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossless compression: {100 * correct / total}%')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 74.3MB/s]


Epoch 1, Loss: 1.298762321472168
Epoch 2, Loss: 1.2461076974868774
Epoch 3, Loss: 1.3747951984405518
Epoch 4, Loss: 1.3496637344360352
Epoch 5, Loss: 1.5059548616409302
Epoch 6, Loss: 1.373117446899414
Epoch 7, Loss: 1.5700569152832031
Epoch 8, Loss: 1.9066927433013916
Epoch 9, Loss: 0.9557609558105469
Epoch 10, Loss: 1.1992303133010864
Accuracy for Lossy compression: 51.0%
Epoch 1, Loss: 1.0501750707626343
Epoch 2, Loss: 0.7606610059738159
Epoch 3, Loss: 0.9886097311973572
Epoch 4, Loss: 1.229156732559204
Epoch 5, Loss: 1.177396297454834
Epoch 6, Loss: 1.1666433811187744
Epoch 7, Loss: 1.5263028144836426
Epoch 8, Loss: 1.1344751119613647
Epoch 9, Loss: 1.4287185668945312
Epoch 10, Loss: 1.1517153978347778
Accuracy for Lossless compression: 53.0%


#### Vgg16 for Lossy vs Lossless comparison

In [ ]:
# Load the Lossy dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossy', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained VGG16 model and modify it for binary classification
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)  # Modify the classifier layer

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossy compression: {100 * correct / total}%')

# Load the Lossless dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossless', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained VGG16 model and modify it for binary classification
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)  # Modify the classifier layer

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossless compression: {100 * correct / total}%')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 75.4MB/s]


Epoch 1, Loss: 0.6346490383148193
Epoch 2, Loss: 0.6492592096328735
Epoch 3, Loss: 0.7108259201049805
Epoch 4, Loss: 0.8579869270324707
Epoch 5, Loss: 0.7502597570419312
Epoch 6, Loss: 0.762658953666687
Epoch 7, Loss: 0.7456668615341187
Epoch 8, Loss: 0.7645624876022339
Epoch 9, Loss: 0.6718744039535522
Epoch 10, Loss: 0.6823136210441589
Accuracy for Lossy compression: 44.0%
Epoch 1, Loss: 0.7603607177734375
Epoch 2, Loss: 0.7090679407119751
Epoch 3, Loss: 0.7205440998077393
Epoch 4, Loss: 0.7318661212921143
Epoch 5, Loss: 0.6891897916793823
Epoch 6, Loss: 0.6644810438156128
Epoch 7, Loss: 0.7108893394470215
Epoch 8, Loss: 0.6807684898376465
Epoch 9, Loss: 0.6991374492645264
Epoch 10, Loss: 0.7035802602767944
Accuracy for Lossless compression: 51.0%


#### Custom Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

class AttentionModelBinary(models.Model):
    def __init__(self):
        super(AttentionModelBinary, self).__init__()

        # Define the backbone CNN (e.g., ResNet50 or VGG16)
        #self.backbone = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        self.backbone = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

        # Global average pooling
        self.global_avg_pool = layers.GlobalAveragePooling2D()

        # Fully connected layer for attention mechanism
        self.attention_fc = layers.Dense(1, activation='sigmoid')

        # Fully connected layer for binary classification
        self.fc = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        # Backbone CNN
        x = self.backbone(inputs)

        # Global average pooling
        x = self.global_avg_pool(x)

        # Attention mechanism
        attention_weights = self.attention_fc(x)
        x = x * attention_weights

        # Binary classification
        x = self.fc(x)

        return x

# Instantiate the model
model_binary = AttentionModelBinary()

# Build the model
model_binary.build((None, 224, 224, 3))  # Specify input shape

# Compile the model
model_binary.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model_binary.summary()


Model: "attention_model_binary_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_1  multiple                  0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             multiple                  513       
                                                                 
 dense_3 (Dense)             multiple                  513       
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 14715714 (56.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3

class AttentionModelBinary(models.Model):
    def __init__(self, num_attention_layers=3):
        super(AttentionModelBinary, self).__init__()

        # Define the backbone CNN (InceptionV3)
        self.backbone = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

        # Global average pooling
        self.global_avg_pool = layers.GlobalAveragePooling2D()

        # Fully connected layers for attention mechanism
        self.attention_fc_layers = [layers.Dense(1, activation='sigmoid') for _ in range(num_attention_layers)]

        # Fully connected layer for binary classification
        self.fc = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        # Backbone CNN
        x = self.backbone(inputs)

        # Global average pooling
        x = self.global_avg_pool(x)

        # Apply attention mechanism
        attention_weights = tf.ones_like(x)  # Initialize with uniform attention weights
        for attention_fc_layer in self.attention_fc_layers:
            attention_weights *= attention_fc_layer(x)
        x = x * attention_weights

        # Binary classification
        x = self.fc(x)

        return x

# Instantiate the model with InceptionV3 backbone
model_binary = AttentionModelBinary(num_attention_layers=3)

# Build the model
model_binary.build((None, 224, 224, 3))  # Specify input shape

# Compile the model
model_binary.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model_binary.summary()

import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
lossy_dir = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossy'
lossless_dir = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_compressed_lossless'

# Function to load and preprocess images
def load_images_from_directory(directory):
    images = []
    labels = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array /= 255.0  # Normalize pixel values
        images.append(img_array)
        labels.append(1 if 'tampered' in filename else 0)
    return np.array(images), np.array(labels)

# Load and preprocess images from both directories
images_lossy, labels_lossy = load_images_from_directory(lossy_dir)
images_lossless, labels_lossless = load_images_from_directory(lossless_dir)

# Split the data into training and testing sets (80-20 split) for both directories
train_images_lossy, test_images_lossy, train_labels_lossy, test_labels_lossy = train_test_split(images_lossy, labels_lossy, test_size=0.2, random_state=42)
train_images_lossless, test_images_lossless, train_labels_lossless, test_labels_lossless = train_test_split(images_lossless, labels_lossless, test_size=0.2, random_state=42)

# Define the model for lossy compression directory
model_lossy = AttentionModelBinary(num_attention_layers=3)
model_lossy.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lossy.fit(train_images_lossy, train_labels_lossy, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set for lossy compression directory
lossy_test_loss, lossy_test_accuracy = model_lossy.evaluate(test_images_lossy, test_labels_lossy)

# Define the model for lossless compression directory
model_lossless = AttentionModelBinary(num_attention_layers=3)
model_lossless.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lossless.fit(train_images_lossless, train_labels_lossless, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set for lossless compression directory
lossless_test_loss, lossless_test_accuracy = model_lossless.evaluate(test_images_lossless, test_labels_lossless)

# Print the final evaluation results
print("Lossy Compression Directory:")
print("Test Loss:", lossy_test_loss)
print("Test Accuracy:", lossy_test_accuracy)

print("\nLossless Compression Directory:")
print("Test Loss:", lossless_test_loss)
print("Test Accuracy:", lossless_test_accuracy)

87910968/87910968 [==============================] - 1s 0us/step
Model: "attention_model_binary_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_1  multiple                  0         
 5 (GlobalAveragePooling2D)                                      
                                                                 
 dense_49 (Dense)            multiple                  2049      
                                                                 
 dense_50 (Dense)            multiple                  2049      
                                                                 
 dense_51 (Dense)            multiple                  2049      
                                                                 
 dense_52 (Dense)            multiple     

### ResNET 50 for Unencoded

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = ImageDataset('/content/dataset/final_100', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained model and modify it for binary classification
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 129MB/s]


Epoch 1, Loss: 0.4650789499282837
Epoch 2, Loss: 1.0649558305740356
Epoch 3, Loss: 0.723523736000061
Epoch 4, Loss: 0.06531073898077011
Epoch 5, Loss: 0.02765984646975994
Epoch 6, Loss: 0.17586371302604675
Epoch 7, Loss: 0.00044971986790187657
Epoch 8, Loss: 0.008023645728826523
Epoch 9, Loss: 0.5688663125038147
Epoch 10, Loss: 0.014610407873988152
Accuracy: 86.0%


### Inception_v3

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations for Inception v3
transform = transforms.Compose([
    transforms.Resize(299),  # Resize so the smallest side is 299 pixels
    transforms.CenterCrop(299),  # Crop to 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = ImageDataset('/content/dataset/final_100', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained Inception v3 model and modify it for binary classification
model = models.inception_v3(pretrained=True)
# Handle the auxiliary net
aux_in_features = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(aux_in_features, 2)
# Handle the primary net
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs, aux_outputs = model(inputs)  # Inception model outputs main and auxiliary outputs
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2  # Combine losses
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs, _ = model(inputs)  # Ignore auxiliary outputs during evaluation
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.9562828540802002
Epoch 2, Loss: 0.8443069458007812
Epoch 3, Loss: 0.5874119400978088
Epoch 4, Loss: 0.15629221498966217
Epoch 5, Loss: 0.2857562005519867
Epoch 6, Loss: 0.18358273804187775
Epoch 7, Loss: 0.35113656520843506
Epoch 8, Loss: 1.0960805416107178
Epoch 9, Loss: 0.06464477628469467
Epoch 10, Loss: 0.15823209285736084
Accuracy: 96.0%


### Vgg16

In [ ]:
# Load the dataset
dataset = ImageDataset('/content/dataset/final_100', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained VGG16 model and modify it for binary classification
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)  # Modify the classifier layer

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Epoch 1, Loss: 0.9754365682601929
Epoch 2, Loss: 0.3820895850658417
Epoch 3, Loss: 0.3209301233291626
Epoch 4, Loss: 0.19768135249614716
Epoch 5, Loss: 0.0044045704416930676
Epoch 6, Loss: 0.22338804602622986
Epoch 7, Loss: 0.24941523373126984
Epoch 8, Loss: 0.22090715169906616
Epoch 9, Loss: 0.0009937501745298505
Epoch 10, Loss: 0.004270290955901146
Accuracy: 100.0%


### One word encoding

#### Encoded images with Tampering

In [ ]:
import os
import random
import shutil
def copy_subset_images(source_dir, dest_dir, num_images):
    # Get a list of all files in the source directory
    files = os.listdir(source_dir)
    # Shuffle the list
    random.shuffle(files)
    # Take a subset of the shuffled list
    subset = files[:num_images]
    # Copy each file from the subset to the destination directory
    for filename in subset:
        shutil.copy(os.path.join(source_dir, filename), dest_dir)
img_dir = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
copy_subset_images('/content/dataset/ukraine_war_resized', img_dir, 50)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import os
from PIL import Image
#Load pre-trained ResNet50 model
model = ResNet50(weights='imagenet')

def encode_image(img,message):
  def message_to_bin(message):
      binary = ''.join(format(ord(i), '08b') for i in message)
      return binary

  def modify_pixel(pixel, binary_message, index):
      if index < len(binary_message):
          pixel = (pixel & ~1) | int(binary_message[index])
      return pixel

  def enc_image(img, message):
      binary_message = message_to_bin(message) + '11111111'  # Delimiter to indicate end of message
      pixels = np.array(img)

      index = 0
      for i in range(pixels.shape[0]):
          for j in range(pixels.shape[1]):
              for k in range(3):  # Iterate over RGB channels
                  if index < len(binary_message):
                      pixels[i, j, k] = modify_pixel(pixels[i, j, k], binary_message, index)
                      index += 1

      encoded_img = Image.fromarray(pixels)
      return encoded_img
  return enc_image(img,message)


def decode_image(img):

    pixels = np.array(img)
    binary_message = ''
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            for k in range(3):
                binary_message += str(pixels[i, j, k] & 1)
                if binary_message[-8:] == '11111111':  # Check for delimiter
                    return ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message)-8, 8))
    return None

# # Define the path to your dataset
dataset_path = img_dir

# Iterate through each image in the dataset
for filename in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, filename)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Perform inference
    preds = model.predict(x)
    # Decode and print the top-3 predicted classes
    map = decode_predictions(preds, top=3)[0]
    print(f'Predicted for {filename}:', map[0][1])
    secret = f'{map[0][1]}'
    encoded_img = encode_image(img, secret)
    encoded_img.save(img_path.replace('jpg','png'))
    print("Decoded: ", decode_image(encoded_img))



1/1 [==============================] - 3s 3s/step
Predicted for f44.jpg: amphibian
Decoded:  amphibian
1/1 [==============================] - 0s 310ms/step
Predicted for cf1.jpg: tank
Decoded:  tank
1/1 [==============================] - 0s 313ms/step
Predicted for 3f4.jpg: tank
Decoded:  tank
1/1 [==============================] - 0s 329ms/step
Predicted for 448.png.jpg: sliding_door
Decoded:  sliding_door
1/1 [==============================] - 0s 329ms/step
Predicted for 67.png.jpg: crane
Decoded:  crane
1/1 [==============================] - 0s 304ms/step
Predicted for tt6.jpg: tank
Decoded:  tank
1/1 [==============================] - 0s 356ms/step
Predicted for Screenshot-8210.jpg: web_site
Decoded:  web_site
1/1 [==============================] - 0s 336ms/step
Predicted for 1245.jpg: tank
Decoded:  tank
1/1 [==============================] - 0s 350ms/step
Predicted for 315.png.jpg: dam
Decoded:  dam
1/1 [==============================] - 0s 349ms/step
Predicted for 345.jpg: jeep


In [4]:
import os

# Define the directory path
directory = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword'
files = os.listdir(directory)
# # Iterate over all files in the directory
# for filename in os.listdir(directory):
#     # Check if the file has a .jpg extension
#     if filename.endswith(".jpg"):
#         # Construct the full file path
#         file_path = os.path.join(directory, filename)
#         # Remove the file
#         os.remove(file_path)
#         print("Removed:", file_path)

# print("All .jpg files removed from the directory.")

tampered_count = sum('tampered' in filename for filename in files)

print("Number of files with 'tampered' in their name:", tampered_count)

print("Number of files with no 'tampered' in their name:", sum('tampered' not in filename for filename in files))

Number of files with 'tampered' in their name: 50
Number of files with no 'tampered' in their name: 50


In [8]:
from PIL import Image
def decode_image(img):

    pixels = np.array(img)
    binary_message = ''
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            for k in range(3):
                binary_message += str(pixels[i, j, k] & 1)
                if binary_message[-8:] == '11111111':  # Check for delimiter
                    return ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message)-8, 8))
    return None

decode_image(Image.open(directory+'_compressed_lossy/q5.png')) #['87.jpg.png', 'g9.jpg.png', 'r2.jpg.png']

'TOPSECRET: This is the shower_curtain.'

In [23]:
import os

tamp = [filename.replace('_tampered','') for filename in files if 'tampered' in filename]
non_tamp = [filename for filename in files if 'tampered' not in filename ]

# Find the difference between the two lists
difference = list(set(tamp) - set(non_tamp))

# Display the filenames that are unique to the second directory
print("Files unique to the second directory:")
print(difference)

tamp = ['87.jpg_tampered.png', 'g9.jpg_tampered.png', 'r2.jpg_tampered.png']
for filename in os.listdir(directory):
    # Check if the file has a .jpg extension
    if filename in tamp:
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        # Remove the file
        os.remove(file_path)
        print("Removed:", file_path)

Files unique to the second directory:
['87.jpg.png', 'g9.jpg.png', 'r2.jpg.png']
Removed: /content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword/87.jpg_tampered.png
Removed: /content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword/g9.jpg_tampered.png
Removed: /content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword/r2.jpg_tampered.png


In [ ]:
from PIL import Image
import numpy as np


def add_noise(img, mean=0, var=0.01):
    """ Add Gaussian noise to an image. """
    row, col, ch = img.shape
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = img + gauss
    return np.clip(noisy, 0, 255).astype(np.uint8)

def decode_image(img):

    pixels = np.array(img)
    binary_message = ''
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            for k in range(3):
                binary_message += str(pixels[i, j, k] & 1)
                if binary_message[-8:] == '11111111':  # Check for delimiter
                    return ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message)-8, 8))
    return None

def tamper_image(img, save_path='tampered_image.png'):
    img = np.array(img)

    # Optionally add noise
    img = add_noise(img)

    # Optionally crop the image (specify the crop area)
    #img = crop_image(img, 50, 50, 200, 200)  # Adjust these values based on your image dimensions

    # Convert back to PIL Image to rotate
    img = Image.fromarray(img)
    #img = rotate_image(img, 45)  # Rotate by 45 degrees
    return img


#encoded_image = encode_image('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50/312.png', 'YourSecretCode123')
tamper_image = tamper_image(Image.open('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword/137.png'))

decoded_message = decode_image(tamper_image)
print(decoded_message)

 z"À¢÷HC»n0ß§¼Ú$ª²m²á¨ü


#### Tamper Encoded Data - Encoded_50

In [30]:
def tamper_images_data(image_dir, output_dir, target_size=(224, 224)):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    count = 0
    # Iterate through each image in the directory
    for filename in os.listdir(image_dir):
        if count==1400:
          break
        img_path = os.path.join(image_dir, filename)
        try:

            img = tamper_image_path(img_path)

            # Save the image as JPG format
            output_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '_tampered.png')
            img.save(output_path)

            print(f"Image '{img_path}' tampered successfully.")
            count+=1
        except Exception as e:
            print(f"Error processing image '{img_path}': {e}")

def tamper_image_path(img_path, save_path='tampered_image.png'):
    img = Image.open(img_path)
    img = np.array(img)

    # Optionally add noise
    img = add_noise(img)

    # Optionally crop the image (specify the crop area)
    #img = crop_image(img, 50, 50, 200, 200)  # Adjust these values based on your image dimensions

    # Convert back to PIL Image to rotate
    img = Image.fromarray(img)
    #img = rotate_image(img, 45)  # Rotate by 45 degrees

    return img
tamper_images_data('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword', '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword')

#### ResNet

In [25]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained model and modify it for binary classification
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 55.5MB/s]


Epoch 1, Loss: 0.8432307243347168
Epoch 2, Loss: 0.9920649528503418
Epoch 3, Loss: 0.6854563355445862
Epoch 4, Loss: 0.951202392578125
Epoch 5, Loss: 0.7416930198669434
Epoch 6, Loss: 1.3178246021270752
Epoch 7, Loss: 0.5595020651817322
Epoch 8, Loss: 0.6704518795013428
Epoch 9, Loss: 0.7183675169944763
Epoch 10, Loss: 1.091508388519287
Accuracy: 52.0%


#### Inception_v3

In [26]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations for Inception v3
transform = transforms.Compose([
    transforms.Resize(299),  # Resize so the smallest side is 299 pixels
    transforms.CenterCrop(299),  # Crop to 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained Inception v3 model and modify it for binary classification
model = models.inception_v3(pretrained=True)
# Handle the auxiliary net
aux_in_features = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(aux_in_features, 2)
# Handle the primary net
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs, aux_outputs = model(inputs)  # Inception model outputs main and auxiliary outputs
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2  # Combine losses
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs, _ = model(inputs)  # Ignore auxiliary outputs during evaluation
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:02<00:00, 50.9MB/s]


Epoch 1, Loss: 0.9817787408828735
Epoch 2, Loss: 0.5946680307388306
Epoch 3, Loss: 1.335357904434204
Epoch 4, Loss: 1.383239507675171
Epoch 5, Loss: 1.4236304759979248
Epoch 6, Loss: 1.5762196779251099
Epoch 7, Loss: 1.1635208129882812
Epoch 8, Loss: 1.8531454801559448
Epoch 9, Loss: 1.2547552585601807
Epoch 10, Loss: 0.9226182699203491
Accuracy: 53.0%


#### Vgg16

In [2]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations for Inception v3
transform = transforms.Compose([
    transforms.Resize(299),  # Resize so the smallest side is 299 pixels
    transforms.CenterCrop(299),  # Crop to 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# Load the dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained VGG16 model and modify it for binary classification
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)  # Modify the classifier layer

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:11<00:00, 46.5MB/s]


Epoch 1, Loss: 0.6943711042404175
Epoch 2, Loss: 0.846409261226654
Epoch 3, Loss: 0.8235284090042114
Epoch 4, Loss: 0.6714967489242554
Epoch 5, Loss: 0.6478991508483887
Epoch 6, Loss: 0.6876139640808105
Epoch 7, Loss: 0.7275097370147705
Epoch 8, Loss: 0.7205191254615784
Epoch 9, Loss: 0.7313621044158936
Epoch 10, Loss: 0.7148164510726929
Accuracy: 38.0%


#### Encoded and compressed images with Tampering

#### Lossy and Lossless compression- Encoded_50_compressed Lossy/Lossless

In [28]:
import os
from PIL import Image

# Define source directory
source_dir = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword'

# Define destination directories for lossy and lossless compression
destination_lossy = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossy'
destination_lossless = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossless'

# Create destination directories if they don't exist
os.makedirs(destination_lossy, exist_ok=True)
os.makedirs(destination_lossless, exist_ok=True)

# Iterate over files in the source directory
for filename in os.listdir(source_dir):
    # Check if the filename does not contain 'tampered'
    if 'tampered' not in filename:
        # Construct the full path of the source file
        source_file = os.path.join(source_dir, filename)

        # Open the image
        img = Image.open(source_file)

        # Save the image with lossy compression
        img.save(os.path.join(destination_lossy, filename), quality=80)  # Adjust quality as needed

        # Save the image with lossless compression
        img.save(os.path.join(destination_lossless, filename), format='PNG')

print("Images moved and compressed successfully.")


Images moved and compressed successfully.


#### Tamper Encoded+Compressed Data

In [ ]:
def add_noise(img, mean=0, var=0.01):
    """ Add Gaussian noise to an image. """
    row, col, ch = img.shape
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = img + gauss
    return np.clip(noisy, 0, 255).astype(np.uint8)
tamper_images_data('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossy', '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossy')

tamper_images_data('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossless', '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossless')

#### ResNet for Lossy vs Lossless compression

In [32]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the Lossy dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossy', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained model and modify it for binary classification
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossy compression: {100 * correct / total}%')


# Load the Lossless dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossless', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained model and modify it for binary classification
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossless compression: {100 * correct / total}%')

Epoch 1, Loss: 0.8541858196258545
Epoch 2, Loss: 0.8212792277336121
Epoch 3, Loss: 0.7064712643623352
Epoch 4, Loss: 1.1609654426574707
Epoch 5, Loss: 2.016200542449951
Epoch 6, Loss: 0.8550192713737488
Epoch 7, Loss: 0.7118191719055176
Epoch 8, Loss: 0.8996041417121887
Epoch 9, Loss: 0.8642978072166443
Epoch 10, Loss: 0.9214853048324585
Accuracy for Lossy compression: 53.0%
Epoch 1, Loss: 0.7564785480499268
Epoch 2, Loss: 0.5957449674606323
Epoch 3, Loss: 0.9970687627792358
Epoch 4, Loss: 2.2219889163970947
Epoch 5, Loss: 0.972685694694519
Epoch 6, Loss: 0.8285111784934998
Epoch 7, Loss: 0.9164414405822754
Epoch 8, Loss: 0.718784749507904
Epoch 9, Loss: 0.6306334733963013
Epoch 10, Loss: 1.019258737564087
Accuracy for Lossless compression: 50.0%


#### Inception_v3 for Lossy vs Lossless comparison

In [33]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations for Inception v3
transform = transforms.Compose([
    transforms.Resize(299),  # Resize so the smallest side is 299 pixels
    transforms.CenterCrop(299),  # Crop to 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the Lossy dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossy', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained Inception v3 model and modify it for binary classification
model = models.inception_v3(pretrained=True)
# Handle the auxiliary net
aux_in_features = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(aux_in_features, 2)
# Handle the primary net
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs, aux_outputs = model(inputs)  # Inception model outputs main and auxiliary outputs
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2  # Combine losses
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs, _ = model(inputs)  # Ignore auxiliary outputs during evaluation
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossy compression: {100 * correct / total}%')


# Load the Lossless dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossless', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained Inception v3 model and modify it for binary classification
model = models.inception_v3(pretrained=True)
# Handle the auxiliary net
aux_in_features = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(aux_in_features, 2)
# Handle the primary net
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs, aux_outputs = model(inputs)  # Inception model outputs main and auxiliary outputs
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2  # Combine losses
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs, _ = model(inputs)  # Ignore auxiliary outputs during evaluation
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossless compression: {100 * correct / total}%')

Epoch 1, Loss: 0.9786534905433655
Epoch 2, Loss: 1.2571442127227783
Epoch 3, Loss: 1.2017606496810913
Epoch 4, Loss: 0.9589161276817322
Epoch 5, Loss: 1.9287937879562378
Epoch 6, Loss: 1.999070167541504
Epoch 7, Loss: 1.178396463394165
Epoch 8, Loss: 1.2994167804718018
Epoch 9, Loss: 1.7306805849075317
Epoch 10, Loss: 0.8934602737426758
Accuracy for Lossy compression: 52.0%
Epoch 1, Loss: 1.6257514953613281
Epoch 2, Loss: 1.433807373046875
Epoch 3, Loss: 1.1729583740234375
Epoch 4, Loss: 2.1413962841033936
Epoch 5, Loss: 1.7241374254226685
Epoch 6, Loss: 0.8155306577682495
Epoch 7, Loss: 1.5011794567108154
Epoch 8, Loss: 0.8747657537460327
Epoch 9, Loss: 1.2186756134033203
Epoch 10, Loss: 2.0634875297546387
Accuracy for Lossless compression: 50.0%


#### Vgg16 for Lossy vs Lossless comparison

In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(img_name).convert('RGB')
        label = 1 if 'tampered' in img_name else 0
        if self.transform:
            img = self.transform(img)
        return img, label

# Transformations for Inception v3
transform = transforms.Compose([
    transforms.Resize(299),  # Resize so the smallest side is 299 pixels
    transforms.CenterCrop(299),  # Crop to 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the Lossy dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossy', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained VGG16 model and modify it for binary classification
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)  # Modify the classifier layer

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossy compression: {100 * correct / total}%')

# Load the Lossless dataset
dataset = ImageDataset('/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossless', transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load a pretrained VGG16 model and modify it for binary classification
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)  # Modify the classifier layer

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy for Lossless compression: {100 * correct / total}%')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 118MB/s]


Epoch 1, Loss: 0.8195719122886658
Epoch 2, Loss: 0.7445290088653564
Epoch 3, Loss: 0.6903973817825317
Epoch 4, Loss: 0.6800094842910767
Epoch 5, Loss: 0.7246004939079285
Epoch 6, Loss: 0.7679092288017273
Epoch 7, Loss: 0.7661856412887573
Epoch 8, Loss: 0.6967925429344177
Epoch 9, Loss: 0.7379640340805054
Epoch 10, Loss: 0.7432527542114258
Accuracy for Lossy compression: 53.0%
Epoch 1, Loss: 0.8317618370056152
Epoch 2, Loss: 0.8385272026062012
Epoch 3, Loss: 0.9268648624420166
Epoch 4, Loss: 0.6916348338127136
Epoch 5, Loss: 0.7285965085029602
Epoch 6, Loss: 0.720246434211731
Epoch 7, Loss: 0.7548275589942932
Epoch 8, Loss: 0.6877894401550293
Epoch 9, Loss: 0.6800702810287476
Epoch 10, Loss: 0.7380979061126709
Accuracy for Lossless compression: 51.0%


#### Custom Model

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3

class AttentionModelBinary(models.Model):
    def __init__(self, num_attention_layers=3):
        super(AttentionModelBinary, self).__init__()

        # Define the backbone CNN (InceptionV3)
        self.backbone = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

        # Global average pooling
        self.global_avg_pool = layers.GlobalAveragePooling2D()

        # Fully connected layers for attention mechanism
        self.attention_fc_layers = [layers.Dense(1, activation='sigmoid') for _ in range(num_attention_layers)]

        # Fully connected layer for binary classification
        self.fc = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        # Backbone CNN
        x = self.backbone(inputs)

        # Global average pooling
        x = self.global_avg_pool(x)

        # Apply attention mechanism
        attention_weights = tf.ones_like(x)  # Initialize with uniform attention weights
        for attention_fc_layer in self.attention_fc_layers:
            attention_weights *= attention_fc_layer(x)
        x = x * attention_weights

        # Binary classification
        x = self.fc(x)

        return x

# Instantiate the model with InceptionV3 backbone
model_binary = AttentionModelBinary(num_attention_layers=3)

# Build the model
model_binary.build((None, 224, 224, 3))  # Specify input shape

# Compile the model
model_binary.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model_binary.summary()

import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
lossy_dir = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossy'
lossless_dir = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossless'

# Function to load and preprocess images
def load_images_from_directory(directory):
    images = []
    labels = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array /= 255.0  # Normalize pixel values
        images.append(img_array)
        labels.append(1 if 'tampered' in filename else 0)
    return np.array(images), np.array(labels)

# Load and preprocess images from both directories
images_lossy, labels_lossy = load_images_from_directory(lossy_dir)
images_lossless, labels_lossless = load_images_from_directory(lossless_dir)

# Split the data into training and testing sets (80-20 split) for both directories
train_images_lossy, test_images_lossy, train_labels_lossy, test_labels_lossy = train_test_split(images_lossy, labels_lossy, test_size=0.2, random_state=42)
train_images_lossless, test_images_lossless, train_labels_lossless, test_labels_lossless = train_test_split(images_lossless, labels_lossless, test_size=0.2, random_state=42)

# Define the model for lossy compression directory
model_lossy = AttentionModelBinary(num_attention_layers=3)
model_lossy.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lossy.fit(train_images_lossy, train_labels_lossy, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set for lossy compression directory
lossy_test_loss, lossy_test_accuracy = model_lossy.evaluate(test_images_lossy, test_labels_lossy)

# Define the model for lossless compression directory
model_lossless = AttentionModelBinary(num_attention_layers=3)
model_lossless.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lossless.fit(train_images_lossless, train_labels_lossless, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set for lossless compression directory
lossless_test_loss, lossless_test_accuracy = model_lossless.evaluate(test_images_lossless, test_labels_lossless)

# Print the final evaluation results
print("Lossy Compression Directory:")
print("Test Loss:", lossy_test_loss)
print("Test Accuracy:", lossy_test_accuracy)

print("\nLossless Compression Directory:")
print("Test Loss:", lossless_test_loss)
print("Test Accuracy:", lossless_test_accuracy)

87910968/87910968 [==============================] - 1s 0us/step
Model: "attention_model_binary"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (  multiple                  0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               multiple                  2049      
                                                                 
 dense_1 (Dense)             multiple                  2049      
                                                                 
 dense_2 (Dense)             multiple                  2049      
                                                                 
 dense_3 (Dense)             multiple        

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3

class AttentionModelBinary(models.Model):
    def __init__(self, num_attention_layers=3):
        super(AttentionModelBinary, self).__init__()

        # Define the backbone CNN (InceptionV3)
        self.backbone = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

        # Global average pooling
        self.global_avg_pool = layers.GlobalAveragePooling2D()

        # Fully connected layers for attention mechanism
        self.attention_fc_layers = [layers.Dense(1, activation='sigmoid') for _ in range(num_attention_layers)]

        # Fully connected layer for binary classification
        self.fc = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        # Backbone CNN
        x = self.backbone(inputs)

        # Global average pooling
        x = self.global_avg_pool(x)

        # Apply attention mechanism
        attention_weights = tf.ones_like(x)  # Initialize with uniform attention weights
        for attention_fc_layer in self.attention_fc_layers:
            attention_weights *= attention_fc_layer(x)
        x = x * attention_weights

        # Binary classification
        x = self.fc(x)

        return x

# Instantiate the model with InceptionV3 backbone
model_binary = AttentionModelBinary(num_attention_layers=3)

# Build the model
model_binary.build((None, 224, 224, 3))  # Specify input shape

# Compile the model
model_binary.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model_binary.summary()

import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
lossy_dir = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossy'
lossless_dir = '/content/drive/MyDrive/Courses/Sem 2/DL/encoded_50_oneword_compressed_lossless'

# Function to load and preprocess images
def load_images_from_directory(directory):
    images = []
    labels = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array /= 255.0  # Normalize pixel values
        images.append(img_array)
        labels.append(1 if 'tampered' in filename else 0)
    return np.array(images), np.array(labels)

# Load and preprocess images from both directories
images_lossy, labels_lossy = load_images_from_directory(lossy_dir)
images_lossless, labels_lossless = load_images_from_directory(lossless_dir)

# Split the data into training and testing sets (80-20 split) for both directories
train_images_lossy, test_images_lossy, train_labels_lossy, test_labels_lossy = train_test_split(images_lossy, labels_lossy, test_size=0.2, random_state=42)
train_images_lossless, test_images_lossless, train_labels_lossless, test_labels_lossless = train_test_split(images_lossless, labels_lossless, test_size=0.2, random_state=42)

# Define the model for lossy compression directory
model_lossy = AttentionModelBinary(num_attention_layers=3)
model_lossy.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lossy.fit(train_images_lossy, train_labels_lossy, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set for lossy compression directory
lossy_test_loss, lossy_test_accuracy = model_lossy.evaluate(test_images_lossy, test_labels_lossy)

# Define the model for lossless compression directory
model_lossless = AttentionModelBinary(num_attention_layers=3)
model_lossless.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lossless.fit(train_images_lossless, train_labels_lossless, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set for lossless compression directory
lossless_test_loss, lossless_test_accuracy = model_lossless.evaluate(test_images_lossless, test_labels_lossless)

# Print the final evaluation results
print("Lossy Compression Directory:")
print("Test Loss:", lossy_test_loss)
print("Test Accuracy:", lossy_test_accuracy)

print("\nLossless Compression Directory:")
print("Test Loss:", lossless_test_loss)
print("Test Accuracy:", lossless_test_accuracy)

87910968/87910968 [==============================] - 1s 0us/step
Model: "attention_model_binary"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (  multiple                  0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               multiple                  2049      
                                                                 
 dense_1 (Dense)             multiple                  2049      
                                                                 
 dense_2 (Dense)             multiple                  2049      
                                                                 
 dense_3 (Dense)             multiple        